# Supersymmetric harmonic oscillator

As a warmup, lets go through the steps to get from the Hamiltonian of this system to a quantum circuit that lets us evolve states according to the time evolution operator.

The superpotential is 
\begin{equation}
    W(\hat{q}) = \frac{1}{2}m \hat{q}^2
\end{equation}
which leads to the following Hamiltonian,
\begin{equation}
    H = \frac{1}{2}\left(H_B+H_F\right), \quad 
        H_B=\hat{p}^2+m^2\hat{q}^2, \quad H_F=m\left[\hat{b}^{\dagger},\hat{b}\right].
\end{equation}
This contains two terms, a bosonic piece and a fermionic piece.  For the bosonic part, we could work in the position or momentum basis, but this is just the Hamiltonian for a harmonic oscillator, so lets convert to the number basis.  


In [2]:
import sys
sys.path.append('..')
from src.sympy_utilities import *
from src.BinaryEncodings import *
from src.MatrixToPauliString import *

import sympy as sp

h_b = 0.5*(p*p + m*m*q*q)

print('Original H = ' + str(h_b))
h_b = sp.expand(h_b.subs(qp_to_ada))
h_f = -0.5*m#[bdag,b]
print('Using creation/annihlation operators\nH_B = ' + str(h_b))
print('H_F = ' + str(h_f))

Original H = 0.5*(m**2*q**2 + p**2)
Using creation/annihlation operators
H_B = 0.5*m*a*ad + 0.5*m*ad*a
H_F = -0.5*m


And we know the matrix elements of $A$ and $A^{\dagger}$, they are defined in HamiltonianTerms.  We of course must impose a cutoff at this point, truncating the allowed excitations of the harmonic oscillator

In [3]:
hamHO = Hamiltonian(h_b, h_f, {m:1, g:1},
                     2, standard_encode)
print('done')
print()
print(hamHO.bmatrix)
print(np.kron(np.eye(2),hamHO.bmatrix))
print(hamHO.bosonPauliStrings)
print()
print(hamHO.fermionic)
print(hamHO.fmatrix)
print(sp.simplify(hamHO.fermionPauliStrings))
print()

print(hamHO.pauliStrings)
print(hamHO.hamMatrix)

done

[[0.500000000000000 0]
 [0 1.50000000000000]]
[[0.500000000000000 0 0 0]
 [0 1.50000000000000 0 0]
 [0 0 0.500000000000000 0]
 [0 0 0 1.50000000000000]]
1.0*I^0 - 0.5*Z^0

-0.500000000000000
[[0.500000000000000 0 0 0]
 [0 0.500000000000000 0 0]
 [0 0 -0.500000000000000 0]
 [0 0 0 -0.500000000000000]]
0.5*I^0*Z^1

1.0*I^0*I^1 + 0.5*I^0*Z^1 - 0.5*I^1*Z^0
[[1.00000000000000 0 0 0]
 [0 2.00000000000000 0 0]
 [0 0 0 0]
 [0 0 0 1.00000000000000]]


In [3]:
hamHO = Hamiltonian(h_b, h_f, {m:1, g:1},
                     3, standard_encode)

print('boson matrix')
print(hamHO.bmatrix)
print('boson matrix as pauli strings')
print(hamHO.bosonPauliStrings)
print('boson pauli strings as matrix')
print(getMatrix(sp.expand(hamHO.bosonPauliStrings)))
print()
print('boson matrix in full hilbert space')
tmpMat=np.kron(np.eye(2),hamHO.bmatrix)
print(tmpMat)
print('boson pauli strings in full hilbert space')
tmpPS=sp.expand(sp.N(mps.matrix_to_pauli_strings(tmpMat, standard_encode)))
print(tmpPS)
print('back to matrix')
print(getMatrix(sp.expand(tmpPS)))
print()
print('fermionic coef')
print(hamHO.fermionic)
print('fermionic matrix')
print(hamHO.fmatrix)
print('fermionic pauli strings')
print(sp.simplify(hamHO.fermionPauliStrings))
print()
print('fermionic pauli strings as Matrix')
print(getMatrix(sp.expand(hamHO.fermionPauliStrings)))

print('Hamiltonian pauli strings')
print(hamHO.pauliStrings)
print('Expected to get...')
tstHam=tmpMat+hamHO.fmatrix
tstPS=sp.expand(mps.matrix_to_pauli_strings(tstHam,standard_encode))
print(tstPS)
print()
print('Ham matrix is')
print(hamHO.hamMatrix)
print('But I would expect it has the same spectrum as')
print(getMatrix(tstPS))

boson matrix
[[0.500000000000000 0 0]
 [0 1.50000000000000 0]
 [0 0 2.50000000000000]]
boson matrix as pauli strings
1.125*I^0*I^1 - 0.125*I^0*Z^1 + 0.375*I^1*Z^0 - 0.875*Z^0*Z^1
boson pauli strings as matrix
[[ 5.00000000e-01+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  1.50000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j  2.50000000e+00+0.j
   0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
  -5.55111512e-17+0.j]]

boson matrix in full hilbert space
[[0.500000000000000 0 0 0 0 0]
 [0 1.50000000000000 0 0 0 0]
 [0 0 2.50000000000000 0 0 0]
 [0 0 0 0.500000000000000 0 0]
 [0 0 0 0 1.50000000000000 0]
 [0 0 0 0 0 2.50000000000000]]
boson pauli strings in full hilbert space
1.125*I^0*I^1*I^2 + 0.125*I^0*I^1*Z^2 + 0.375*I^0*I^2*Z^1 - 0.625*I^0*Z^1*Z^2 + 0.25*I^1*Z^0*Z^2 - 0.5*I^2*Z^0*Z^1 - 0.25*Z^0*Z^1*Z^2
back to matrix
[[ 5.00000000e-01+0.j  0.00000000e+00+0.

# Supersymmetric Anharmonic Oscillator

The superpotential is 
\begin{equation}
    W(\hat{q}) = \frac{1}{2}m\hat{q}^2 + \frac{1}{4}g\hat{q}^4
\end{equation}

The full Hamiltonian is
\begin{equation}
    H=\frac{1}{2}\left[\hat{p}^2 + m^2\hat{q}^2 + 2mg\hat{q}^4 + g^2\hat{q}^6 - (m+3g\hat{q}^2)\left[b^{\dagger},b\right]\right]
\end{equation}

In [4]:
import sys
sys.path.append('..')
from src.sympy_utilities import *
from src.BinaryEncodings import *

import sympy as sp

h_b = 0.5*(p*p + m*m*q*q + 2.*m*g*q*q*q*q + g*g*q*q*q*q*q*q)
h_f = -0.5*(m+3*g*q*q)#[bdag,b]


n=3
hamAHO = Hamiltonian(h_b, h_f, {m:1, g:1},
                    n, standard_encode)

In [5]:
print('boson matrix')
print(hamAHO.bmatrix)
print('boson matrix as pauli strings')
print(hamAHO.bosonPauliStrings)
print('boson pauli strings as matrix')
print(getMatrix(sp.expand(hamAHO.bosonPauliStrings)))
print()
print('boson matrix in full hilbert space')
tmpMat=np.kron(np.eye(2),hamAHO.bmatrix)
print(tmpMat)
print('boson pauli strings in full hilbert space')
tmpPS=sp.expand(sp.N(mps.matrix_to_pauli_strings(tmpMat, standard_encode)))
tmpPS=tmpPS.xreplace(dict([(n,0) for n in tmpPS.atoms(sp.Float) if abs(n) < 1e-12]))
print(tmpPS)
print('back to matrix')
print(getMatrix(sp.expand(tmpPS)))
print()
print('fermionic coef')
print(hamAHO.fermionic)
print('fermionic matrix')
print(hamAHO.fmatrix)
print('fermionic pauli strings')
print(sp.simplify(hamAHO.fermionPauliStrings))
print()
print('fermionic pauli strings as Matrix')
print(getMatrix(sp.expand(hamAHO.fermionPauliStrings)))

print('Hamiltonian pauli strings')
print(hamAHO.pauliStrings)
print('Expected to get...')
tstHam=tmpMat+hamAHO.fmatrix
tstPS=sp.expand(mps.matrix_to_pauli_strings(tstHam,standard_encode))
tstPS=tstPS.xreplace(dict([(n,0) for n in tstPS.atoms(sp.Float) if abs(n) < 1e-12]))
print(tstPS)
print()
print('Ham matrix is')
print(hamAHO.hamMatrix)
print('But I would expect it has the same spectrum as')
print(getMatrix(tstPS))

boson matrix
[[2.18750000000000 0 6.09879598773397]
 [0 11.8125000000000 0]
 [6.09879598773397 0 35.6875000000000]]
boson matrix as pauli strings
12.421875*I^0*I^1 + 3.04939799386699*I^0*X^1 + 2.22044604925031e-16*I*I^0*Y^1 - 5.421875*I^0*Z^1 + 6.515625*I^1*Z^0 + 3.04939799386699*X^1*Z^0 + 2.22044604925031e-16*I*Y^1*Z^0 - 11.328125*Z^0*Z^1
boson pauli strings as matrix
[[ 2.1875    +0.j  0.        +0.j  6.09879599+0.j  0.        +0.j]
 [ 0.        +0.j 11.8125    +0.j  0.        +0.j  0.        +0.j]
 [ 6.09879599+0.j  0.        +0.j 35.6875    +0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j]]

boson matrix in full hilbert space
[[2.18750000000000 0 6.09879598773397 0 0 0]
 [0 11.8125000000000 0 0 0 0]
 [6.09879598773397 0 35.6875000000000 0 0 0]
 [0 0 0 2.18750000000000 0 6.09879598773397]
 [0 0 0 0 11.8125000000000 0]
 [0 0 0 6.09879598773397 0 35.6875000000000]]
boson pauli strings in full hilbert space
12.421875*I^0*I^1*I^2 + 0.546875*I^0*I^1

In [6]:
print(np.linalg.eig(np.array(hamAHO.hamMatrix,dtype=float))[0])
print(np.linalg.eig(np.array(getMatrix(tstPS),dtype=complex))[0])

[ 2.08341082 41.29158918 32.24817597  0.12682403 14.5625      9.0625    ]
[ 2.08341082+0.j 41.29158918+0.j 32.24817597+0.j  0.12682403+0.j
 14.5625    +0.j  9.0625    +0.j  0.        +0.j  0.        +0.j]


In [7]:
h_b = 0.5*(p*p + m*m*q*q + 2.*m*g*q*q*q*q + g*g*q*q*q*q*q*q)
h_f = -0.5*(m+3*g*q*q)#[bdag,b]


n=2
hamAHO = Hamiltonian(h_b, h_f, {m:1, g:0},
                    n, standard_encode)

In [8]:
hamAHO.pauliStrings

1.0*I^0*I^1 + 0.5*I^0*Z^1 - 0.5*I^1*Z^0

In [9]:
h_b = 0.5*(p*p + m*m*q*q + 2.*m*g*q*q*q*q + g*g*q*q*q*q*q*q)
h_f = 0.5*(m+3*g*q*q)#[bdag,b]


n=4
hamAHO = Hamiltonian(h_b, h_f, {m:1, g:1},
                    n, standard_encode)

In [10]:
hamAHO.pauliStrings

32.75*I^0*I^1*I^2 - 3.5*I^0*I^1*Z^2 + 14.1486483908533*I^0*I^2*X^1 - 25.75*I^0*I^2*Z^1 - 1.4488887394336*I^0*X^1*Z^2 + 1.5*I^0*Z^1*Z^2 - 13.8125*I^1*I^2*Z^0 + 0.75*I^1*Z^0*Z^2 - 8.04985240311929*I^2*X^1*Z^0 + 9.0*I^2*Z^0*Z^1 + 0.388228567653781*X^1*Z^0*Z^2

In [11]:
hamAHO.hamMatrix

array([[0.937500000000000, 0, 5.03813581595415, 0, 0, 0, 0, 0],
       [0, 9.06250000000000, 0, 20.3613834868852, 0, 0, 0, 0],
       [5.03813581595415, 0, 31.4375000000000, 0, 0, 0, 0, 0],
       [0, 20.3613834868852, 0, 75.5625000000000, 0, 0, 0, 0],
       [0, 0, 0, 0, 3.43750000000000, 0, 7.15945615951379, 0],
       [0, 0, 0, 0, 0, 14.5625000000000, 0, 24.0356181010599],
       [0, 0, 0, 0, 7.15945615951379, 0, 39.9375000000000, 0],
       [0, 0, 0, 0, 0, 24.0356181010599, 0, 87.0625000000000]],
      dtype=object)

In [12]:
h_b = 0.5*(p*p + m*m*q*q + 2.*m*g*q*q*q*q + g*g*q*q*q*q*q*q)
h_f = 0.5*(m+3*g*q*q)#[bdag,b]


n=16
hamAHO = Hamiltonian(h_b, h_f, {m:1, g:1},
                    n, standard_encode)

In [13]:
hamAHO.hamMatrix

array([[0.937500000000000, 0, 5.03813581595415, ..., 0, 0, 0],
       [0, 9.06250000000000, 0, ..., 0, 0, 0],
       [5.03813581595415, 0, 31.4375000000000, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 3404.56250000000, 0, 3087.56895798499],
       [0, 0, 0, ..., 0, 4185.93750000000, 0],
       [0, 0, 0, ..., 3087.56895798499, 0, 5079.06250000000]],
      dtype=object)

In [14]:
np.linalg.eig(np.array(hamAHO.hamMatrix,dtype=float))

(array([6.57894977e+03, 2.25639928e+03, 8.15513246e+03, 7.72064707e+02,
        2.94732376e+03, 1.07392833e+03, 2.41232814e+02, 3.63855960e+02,
        6.48981985e+01, 1.08552639e+02, 1.55392089e+01, 1.83029849e-03,
        4.41418952e+00, 2.80518338e+01, 1.70732190e+00, 7.94769938e+00,
        6.64516415e+03, 8.22656441e+03, 2.30194206e+03, 2.99741317e+03,
        8.03262342e+02, 1.10905748e+03, 2.61810455e+02, 3.87766690e+02,
        7.75413666e+01, 1.23939075e+02, 2.23264623e+01, 1.68720122e+00,
        7.76596507e+00, 3.70114181e+01, 4.37401524e+00, 1.23737466e+01]),
 array([[-1.03525103e-05,  3.13339444e-04,  2.13081460e-16, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 7.88134159e-17,  1.52831849e-16,  3.02380308e-05, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-3.29898140e-04,  6.84387248e-03,  4.73645581e-18, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        ...,
        [ 0.00000000e+00,  0.00

# Double Well Potential

In [15]:
h_b_DW = 0.5*(p*p + (m*q + g*(q*q + mu*mu))**2)
h_f_DW = -0.5*(m+2*g*q)#[bdag,b]

In [16]:
from src.BinaryEncodings import *

hamDW = Hamiltonian(h_b_DW, h_f_DW, {m:1, g:1, mu:1},
                     2, standard_encode)
print('boson matrix')
print(hamDW.bmatrix)
print('boson matrix as pauli strings')
print(hamDW.bosonPauliStrings)
print('boson pauli strings as matrix')
print(getMatrix(sp.expand(hamDW.bosonPauliStrings)))
print()
print('boson matrix in full hilbert space')
tmpMat=np.kron(np.eye(2),hamDW.bmatrix)
print(tmpMat)
print('boson pauli strings in full hilbert space')
tmpPS=sp.expand(sp.N(mps.matrix_to_pauli_strings(tmpMat, standard_encode)))
tmpPS=tmpPS.xreplace(dict([(n,0) for n in tmpPS.atoms(sp.Float) if abs(n) < 1e-12]))
print(tmpPS)
print('back to matrix')
print(getMatrix(sp.expand(tmpPS)))
print()
print('fermionic coef')
print(hamDW.fermionic)
print('fermionic matrix')
print(hamDW.fmatrix)
print('fermionic pauli strings')
print(sp.simplify(hamDW.fermionPauliStrings))
print()
print('fermionic pauli strings as Matrix')
print(getMatrix(sp.expand(hamDW.fermionPauliStrings)))

print('Hamiltonian pauli strings')
print(hamDW.pauliStrings)
print('Expected to get...')
tstHam=tmpMat+hamDW.fmatrix
tstPS=sp.expand(mps.matrix_to_pauli_strings(tstHam,standard_encode))
tstPS=sp.N(tstPS.xreplace(dict([(n,0) for n in tstPS.atoms(sp.Float) if abs(n) < 1e-12])))
print(tstPS)
print()
print('Ham matrix is')
print(hamDW.hamMatrix)
print('But I would expect it has the same spectrum as')
print(getMatrix(tstPS))

boson matrix
[[1.87500000000000 1.25*sqrt(2)]
 [1.25*sqrt(2) 5.37500000000000]]
boson matrix as pauli strings
3.625*I^0 + 1.76776695296637*X^0 - 1.75*Z^0
boson pauli strings as matrix
[[1.875     +0.j 1.76776695+0.j]
 [1.76776695+0.j 5.375     +0.j]]

boson matrix in full hilbert space
[[1.87500000000000 1.25*sqrt(2) 0 0]
 [1.25*sqrt(2) 5.37500000000000 0 0]
 [0 0 1.87500000000000 1.25*sqrt(2)]
 [0 0 1.25*sqrt(2) 5.37500000000000]]
boson pauli strings in full hilbert space
3.625*I^0*I^1 + 1.76776695296637*I^1*X^0 - 1.75*I^1*Z^0
back to matrix
[[1.875     +0.j 1.76776695+0.j 0.        +0.j 0.        +0.j]
 [1.76776695+0.j 5.375     +0.j 0.        +0.j 0.        +0.j]
 [0.        +0.j 0.        +0.j 1.875     +0.j 1.76776695+0.j]
 [0.        +0.j 0.        +0.j 1.76776695+0.j 5.375     +0.j]]

fermionic coef
-0.5 - 0.5*sqrt(2)*a - 0.5*sqrt(2)*ad
fermionic matrix
[[0.500000000000000 0.5*sqrt(2) 0 0]
 [0.5*sqrt(2) 0.500000000000000 0 0]
 [0 0 -0.500000000000000 -0.5*sqrt(2)]
 [0 0 -0.5*sqr

In [17]:
import math
1.75*math.sqrt(2)

2.4748737341529163

In [18]:
0.75*math.sqrt(2)

1.0606601717798214

## Compare to QISKIT MatrixOp??

In [1]:
import qiskit
from qiskit.opflow import MatrixOp

In [5]:
hamHO.pauliStrings

1.0*I^0*I^1 + 0.5*I^0*Z^1 - 0.5*I^1*Z^0

In [7]:
getMatrix(hamHO.pauliStrings)

array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 2.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j]])

In [6]:
hamHO.hamMatrix

array([[1.00000000000000, 0, 0, 0],
       [0, 2.00000000000000, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 1.00000000000000]], dtype=object)

In [8]:
MatrixOp(hamHO.hamMatrix)

MatrixOp(Operator([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 2.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j]],
         input_dims=(2, 2), output_dims=(2, 2)), coeff=1.0)